In [4]:
pip install qdrant-client --user

  Using cached qdrant_client-1.13.3-py3-none-any.whl (306 kB)
  Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
  Using cached portalocker-2.10.1-py3-none-any.whl (18 kB)
  Using cached grpcio_tools-1.71.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.5 MB)
  Using cached protobuf-5.29.4-cp38-abi3-manylinux2014_x86_64.whl (319 kB)
  Using cached grpcio-1.71.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.9 MB)
  Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
  Using cached h2-4.2.0-py3-none-any.whl (60 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached hyperframe-6.1.0-py3-none-any.whl (13 kB)
  Using cached hpack-4.1.0-py3-none-any.whl (34 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.


In [5]:
from qdrant_client import QdrantClient

print("Attempting to connect to Qdrant at http://qdrant.d2f.io.vn")
try:
    client = QdrantClient(url="http://qdrant.d2f.io.vn")
    print("Connected to Qdrant, recreating collection")
    client.recreate_collection(
        collection_name="test_v2",  # Replace with your COLLECTION_NAME
        vectors_config={"size": 128, "distance": "Cosine"},
    )
except Exception as e:
    print(f"Failed to connect to Qdrant: {str(e)}")

ModuleNotFoundError: No module named 'qdrant_client'